# TODO : Intro

## Install SAP Cloud SDK for AI
For this workshop we will use the SAP Cloud SDK for AI : Python. SAP Cloud SDK for AI is the official SDFk for SAP AI core, generative AI Hub, and orchestration. Overall the SDK composed of three python disctribution:
* You can use **sap-ai-sdk-base** to access the AI API using Python methods and data structures.
* You can use **sap-ai-sdk-core** to interact with SAP AI Core for administration and public lifecycle management. We will need this later to create object store secrets. 
* You can use **sap-ai-sdk-gen** to integrate native SDK libraries and langchain for accessing models and leverage the orchestration service of generative AI hub with capabilities such as templating, grounding, data masking, and content filtering. For more information, see Generative AI Hub.



In [ ]:

! pip install sap-ai-sdk-gen
! pip install sap-ai-sdk-core 

## Warm up: Ground your LLM with SAP Help
Let us get started with a quick form up by grounding you LLM with SAP Help. 

In [1]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.config import GroundingModule, OrchestrationConfig
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding, DocumentGroundingFilter
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.service import OrchestrationService

In [ ]:
llm = LLM(
    name="gemini-2.5-flash",
    parameters={
        'temperature': 0.0,
    }
)
template = Template
            messages=[
                SystemMessage("You are a helpful translation assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

In [3]:
# Set up Document Grounding
filters = [
            DocumentGroundingFilter(id="SAPHelp", data_repository_type="help.sap.com")
        ]

grounding_config = GroundingModule(
            type="document_grounding_service",
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

In [4]:
import importlib

orchestration_service = OrchestrationService(
    api_url="https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d002ccdb23f8195c",
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue( 
            name="user_query",
            value="What roles do I need to access the SAP AI Launchpad ?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

The provided text focuses on access to generative AI features *within* SAP S/4HANA Cloud Public Edition, not SAP AI Launchpad itself.  There's no information about the roles needed to access the SAP AI Launchpad.  The document describes roles and authorizations for various AI-powered features *integrated* with S/4HANA, not the Launchpad's access control.

To determine the necessary roles for accessing SAP AI Launchpad, you should consult the official SAP AI Launchpad documentation or your SAP system administrator.  The information needed is not present in the provided text.



## Ground your LLM with Custom Documents

Besides grounding your LLM with SAP help you can of course ground your LLM with you own documents. The document file that the grounding service support here are PDF, HTML, TXT, JPEG, JPG, DOCX, PNG, TIFF, PPT.
However to use Grounding you need to prepare a knowlegde base in advance. Generative AI hub provides two otions for the users to provide data (prepare knowledege base):

* Option 1: Provide the chunks of document via Vector API directly
* Option 2: Upload the documents to supported data repository and run the data pipeline to vectorize the documents

In the following you will get to know both option, and we start with option 1

### Provide the chunks of documents via Vector API directly

In the following we will do the following steps:
+ Prepare the Vector Knowledge Base
    1. Create Collection
    2. Create documents by directly using the chunks of data provided by users
    3.  Store data in the vector database
    4.  Assign repository IDs to access the data
* Configure Grounding Module in the Orchestration 
    1. Create a grounding request configuration in the orchestration pipeline using repository IDs
    2.  Run the orchestration pipeline and check that the response refers to the user data

In [ ]:
# Or via Bruno 

### Upload the documents to the supported data repository and run data pipeline

#### Create generic Secrets for Object Store 
Generic secrets securely store AWS S3 credentials required fro document access

In [33]:
import logging
 
logging.basicConfig(level=logging.DEBUG)
logging.getLogger("ai_core_sdk").setLevel(logging.DEBUG)

In [34]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import os, json
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import os, json
import base64

#Get the Service Key and Instantiate AI API Client
aic_config_file = "/Users/d054176/Downloads/aicore_service_key.json"
with open(aic_config_file,'rb') as config:
    aic_config_file = json.load(config)
    config.close()

base_url = aic_config_file['serviceurls']['AI_API_URL']
client_id = aic_config_file['clientid']
client_secret = aic_config_file['clientsecret']
auth_url = aic_config_file['url']



client = AICoreV2Client (base_url=base_url+'/v2',
                         auth_url= auth_url+'/oauth/token',
                         client_id=client_id,
                         client_secret=client_secret,
                         resource_group="default"
                         )


os_config_file = "/Users/d054176/Downloads/objectstore_skey.json"
with open(os_config_file,'rb') as config:
    os_config_file = json.load(config)
    config.close()

def b64(val):
     return base64.b64encode(val.encode("utf-8")).decode("utf-8")

access_key = os_config_file['access_key_id']
secret = os_config_file['secret_access_key']
bucket = os_config_file['bucket']
uri = os_config_file['uri']
user = os_config_file['username']

def secret_dict():
        return {
            'name': 'aws3-secret-3',
            'data': {
            "authentication": b64("NoAuthentication"),
            "description": b64("For Grounding"),
            "access_key_id": b64(access_key),
            "bucket": b64(bucket),
            "host": b64("s3-eu-central-1.amazonaws.com"),
            "region": b64("eu-central-1"), 
            "secret_access_key": b64(secret),
            "username": b64(user)            
            },
            "labels": [
                {
                    "key": "ext.ai.sap.com/document-grounding",
                    "value": "true"
                },
                {
                    "key": "ext.ai.sap.com/documentRepositoryType",
                    "value": "S3"
                }
         ]
        }
body = {
    'name': secret_dict()['name'],
    'data': secret_dict()['data'],
    'labels': secret_dict()['labels']
}
headers = {'AI-Tenant-Scope': "false"}

response_dict = client.rest_client.post(path='/admin/secrets', body=body, resource_group="AI167000",
                                        headers=headers)
print(response_dict)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ai167-000-s40ku9xc.authentication.eu10.hana.ondemand.com:443
DEBUG:urllib3.connectionpool:https://ai167-000-s40ku9xc.authentication.eu10.hana.ondemand.com:443 "POST /oauth/token HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com:443
DEBUG:urllib3.connectionpool:https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com:443 "POST /v2/admin/secrets HTTP/1.1" 400 126


AIAPIInvalidRequestException: Failed to post /admin/secrets: Missing required attribute(s): url, access_key_id, secret_access_key 
 Status Code: 400, Request ID:None

#### Create Data Pipeline

In [27]:
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.document_grounding.client import PipelineAPIClient
from gen_ai_hub.document_grounding.models.pipeline import S3PipelineCreateRequest, CommonConfiguration

aicore_client = get_proxy_client();
print(aicore_client.resource_group) 
print(aicore_client.base_url)

pipeline_api_client = PipelineAPIClient(aicore_client)

generic_secret_s3_bucket = "aws3-secret-2"
s3_config = S3PipelineCreateRequest(configuration= CommonConfiguration(destination=generic_secret_s3_bucket))
response = pipeline_api_client.create_pipeline(s3_config)

print(f"Reference the Vector knowledge base using the pipeline ID: {response.pipelineId}")
# check the status of the vectorization pipeline until it is completed
print(pipeline_api_client.get_pipeline_status(response.pipelineId))

None
None


AIAPIServerException: Failed to post /lm/document-grounding/pipelines: {"code": "404", "message": "Received not found from document grounding api = {'request_id': '48ca4423-8340-458d-8cbf-bee41afea7ca', 'code': 404, 'message': 'Pipeline creation failed because the specified destination could not be found. Please verify the destination name'}", "requestId": "9706c21a-aeaa-9dfc-b41a-1a711c8e49bb", "details": [{"code": "404", "message": "Received not found from document grounding api = {'request_id': '48ca4423-8340-458d-8cbf-bee41afea7ca', 'code': 404, 'message': 'Pipeline creation failed because the specified destination could not be found. Please verify the destination name'}"}]} 
 Status Code: 404, Request ID:None

In [17]:
secrets = client.secrets.get(name="aws3-secret")

NotImplementedError: 